<a href="https://colab.research.google.com/github/MaricelaMH/SIMULACION-II/blob/main/Linea_de_espera_combi_proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SIMULACIÓN DE UNA PARADA DE COMBIS

Contexto del problema: En Naucalpan de Juarez, Estado de México, hay una parada de combis en el paradero de Cuatro Caminos donde los pasajeros se forman para esperar la próxima llegada de una combi. Esta parada sigue una disciplina de servicio primero en entrar, primero en salir (FIFO), donde la combi recoge a los pasajeros en el orden en que llegaron. Para ello realizaremos una simulación para lograr entender mejor el comportamiento de nuestro sistema y asi tomar desiciones sobre los horarios y capacidad de las combis para evitar tiempos de esperas excesivos parea los pasajeros.

1. Para simular lo anterior, usaremos la libreria simpy por lo que la importaremos junto con las demas librerias que usaremos.

In [1]:
!pip install simpy

In [21]:
import matplotlib.pyplot as plt
import numpy as np
import random
import simpy

2. Definimos nuestros parámetros

In [44]:
# Parámetros del sistema de combis
no_combis = 1            # Cantidad de combis disponibles en la parada
tiempo_llegada_pasajeros = 2                 # Tasa de llegada de los pasajeros (1 cada 5 minutos)
tiempo_llegada_combis =  20                 # Tasa de llegada de combis (1 cada 20 minutos)
cap_combi = 19              # Capacidad máxima de pasajeros de cada combi
tiempo_prom_abordo = 1/2     # Tiempo promedio de abordo de cada pasajero (en minutos)

In [45]:
# Tiempo de simulación (en minutos)
tiempo_simulacion=120

In [46]:
# Creamos nuestras listas para almacenar nuestros datos
longitud_cola=[]
tiempo_espera=[]
tiempos_abordo=[]
combis_necesarias=[]  # Lista para la cantidad de combis necesarias

3. Creamos una función que nos ayudara a simular la llegada de los pasajeros a la parada de combis las cuales suelen seguir una distribución exponencial, por lo que la media estara definida como:

$$ \mu = \frac{1}{ll}  $$

donde $ll$ es el tiempo promedio de llegada de los pasajeros

In [47]:
# Función para simular cuando los pasajeros llegan a la parada de combis.
def llegada_pasajeros(env, cola_pasajeros,tiempo_pasajero, texto=True):
  numero_pasajero = 1  # Contador para identificar cada pasajero
  while True:
        # Sabiendo que el tiempos de llegada de los pasajeros siguen una distribución exponencial
        tiempo_ll_pas = random.expovariate(1 / tiempo_llegada_pasajeros)
        yield env.timeout(tiempo_ll_pas) #Pausamos la ejecución durante tiempo_ll unidades de tiempo

        pasajero = f'--- El pasajero número {numero_pasajero}'
        print(f'{pasajero} llegó en el minuto {env.now:.2}----')

        cola_pasajeros.put((pasajero,env.now)) #Guardamos el tiempo de llegada de los pasajeros
        tiempo_pasajero.append(env.now) #Guardamos el tiempo de llegada de los pasajeros

        numero_pasajero+=1 #Incrementamos el contador de pasajeros

        # Actualizamos la longitud de la cola
        longitud_cola.append(len(cola_pasajeros.items))

4. Creamos nuevamente otra función que nos ayudara a simular el tiempo de llegadas de las combis

In [48]:
def llegada_combis(env, cola_pasajeros,combis,tiempo_pasajero,texto=True):
   numero_combi = 1  # Contador para identificar cada combi
   while True:
        # El tiempo de llegada sigue una distribución exponencial
        tiempo_ll_com = random.expovariate(1 / tiempo_llegada_combis)
        yield env.timeout(tiempo_ll_com)

        combi = f' La combi número {numero_combi}'
        print(f'{combi} llegó en el minuto {env.now:.2f} ***')
        pasajeros = []
        tiempo_abordo_combi = 0

        # Las personas abordan la combi hasta que se llena
        while len(pasajeros) < cap_combi and len(cola_pasajeros.items) > 0:
            pasajero, tiempo_ll_pas = yield cola_pasajeros.get()  # Persona sube a la combi
            pasajeros.append(pasajero)
            tiempo_abordo_combi += random.expovariate(tiempo_prom_abordo)  # Tiempo de abordo de cada persona (promedio 2 minutos)
            # Registramos el tiempo de espera para esta persona (tiempo de llegada a la cola y tiempo de abordo)
            tiempo_espera_pasajero = env.now - tiempo_ll_pas
            tiempo_espera.append(tiempo_espera_pasajero)
            print(f'{pasajero} subió a  {combi} después de {tiempo_espera_pasajero:.2f} minutos de espera +++')

        # La combi parte con los pasajeros
        combis.put(pasajeros)
        print(f'{combi} se fue con {len(pasajeros)} pasajeros <--.')

        # Registrar el tiempo de abordo de cada persona
        tiempos_abordo.append(tiempo_abordo_combi / len(pasajeros) if pasajeros else 0)

        # Registrar la cantidad de combis necesarias (realmente activas)
        combis_necesarias.append(1)  # Una nueva combi disponible
        numero_combi += 1  # Incrementar el ID de la combi

        # Cuando una combi se va, eliminamos la combi del sistema (por eso estamos usando combis.put)
        if len(pasajeros) > 0:
            combis_necesarias[-1] = len(pasajeros)

5. Creamos una función llamada abordaje la cual ayudara a simular el tiempo que le lleva al pasajero abordar una combi , donde utilizaremos una distribución uniforme para determinar el tiempo mínimo y máximo de abordaje del pasajero.

In [49]:
def simular():
    """Función principal para ejecutar la simulación."""
    env = simpy.Environment()
    cola_pasajeros = simpy.Store(env)  # Cola de personas
    combis = simpy.Store(env)  # Cola de combis
    tiempo_pasajero = []  # Para guardar el tiempo de llegada de cada persona

    # Estimamos el número de combis necesarias
    estimar_num_combis()

    # Arrancamos los procesos de llegada
    env.process(llegada_pasajeros(env, cola_pasajeros, tiempo_pasajero))
    env.process(llegada_combis(env, cola_pasajeros, combis, tiempo_pasajero))

    # Ejecutamos la simulación
    env.run(until=tiempo_simulacion)


In [50]:
# Ejecutamos la simulación
simular()

El número estimado de combis necesarias es de: 3.16
--- El pasajero número 1 llegó en el minuto 1.7----
 La combi número 1 llegó en el minuto 1.75 ***
--- El pasajero número 1 subió a   La combi número 1 después de 0.04 minutos de espera +++
 La combi número 1 se fue con 1 pasajeros <--.
 La combi número 2 llegó en el minuto 2.87 ***
 La combi número 2 se fue con 0 pasajeros <--.
--- El pasajero número 2 llegó en el minuto 3.7----
--- El pasajero número 3 llegó en el minuto 4.2----
--- El pasajero número 4 llegó en el minuto 4.5----
--- El pasajero número 5 llegó en el minuto 5.4----
--- El pasajero número 6 llegó en el minuto 6.5----
--- El pasajero número 7 llegó en el minuto 6.5----
--- El pasajero número 8 llegó en el minuto 6.9----
--- El pasajero número 9 llegó en el minuto 7.6----
--- El pasajero número 10 llegó en el minuto 1e+01----
 La combi número 3 llegó en el minuto 10.54 ***
--- El pasajero número 2 subió a   La combi número 3 después de 6.81 minutos de espera +++
--- El 

In [51]:
def estimar_num_combis(): # Calculamos el numero de combis necesarias
    # Tasa de llegada de personas
    tasa_llegada_personas = 1 / tiempo_llegada_pasajeros
    # Número estimado de combis
    num_combis_estimadas = (tasa_llegada_personas * tiempo_simulacion) / cap_combi
    print(f'El número estimado de combis necesarias es de: {num_combis_estimadas:.2f}')
    return num_combis_estimadas